In [1]:
#import gensim.utils as util
#util.test_local()

from gensim.models import Word2Vec
import logging
import os
from tqdm import tqdm
import sys
import json

from gensim import downloader
text8 = downloader.load('text8')

In [2]:
from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()

def load_json_pairs(input_file):
    with open(input_file, "r") as fp:
        pairs = json.load(fp)
    return pairs
PATH = '/home/manni/bias/counterfactual-data-substitution/cds/data/'
base_pairs = load_json_pairs(PATH+'cda_default_pairs_new.json')
#base_pairs = load_json_pairs('/home/manni/bias/counterfactual-data-substitution/cds/data/cda_default_filtered_pairs.json')
name_pairs = load_json_pairs(PATH+'names_pairs_1000_scaled.json')


for (male, female) in base_pairs:
    # if male == 'he':
    #     continue
    keyword_processor.add_keyword(male.lower(),female.lower())
    keyword_processor.add_keyword(female.lower(),male.lower())

# for (male, female) in name_pairs:
#     keyword_processor.add_keyword(male.lower(),female.lower())
#     keyword_processor.add_keyword(female.lower(),male.lower())

In [3]:
# Set up logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)
print('Cpu count:',os.cpu_count())

Cpu count: 48


In [4]:
def preprocess_text(text):
    text = ' '.join(text)
    text = text.lower()
    text_ = keyword_processor.replace_keywords(text)
    return text_.split()+text.split()


class SentenceIterator:
    def __iter__(self):
        for document in text8:
            _document = preprocess_text(document)
            yield _document

sentences = SentenceIterator()

In [5]:
# Initialize and train the Word2Vec model
model = Word2Vec(sentences=sentences, shrink_windows=True, sg=1, vector_size=300, window=5, min_count=3, workers=os.cpu_count())

2023-11-14 13:18:30,806 : INFO : collecting all words and their counts
2023-11-14 13:18:30,808 : DEBUG : {'uri': '/home/manni/gensim-data/text8/text8.gz', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
2023-11-14 13:18:30,866 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-11-14 13:19:06,327 : INFO : collected 253906 word types from a corpus of 34010414 raw words and 1701 sentences
2023-11-14 13:19:06,331 : INFO : Creating a fresh vocabulary
2023-11-14 13:19:06,816 : DEBUG : starting a new internal lifecycle event log for Word2Vec
2023-11-14 13:19:06,817 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=3 retains 135394 unique words (53.32% of original 253906, drops 118512)', 'datetime': '2023-11-14T13:19:06.816764', 'gensim': '4.3.2.dev0', 'python': '3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]', 'platfo

In [6]:
# Save the trained model
vocab = model.wv.key_to_index
emb_file = '/home/manni/embs/CDA_en_text8_mc3_epoch5_300.txt'
logging.info('Save trained word vectors')
with open(emb_file, 'w', encoding='utf-8') as f:
    f.write('%d %d\n' % (len(vocab), 300)) 
    for word in tqdm(vocab, position=0):
        f.write('%s %s\n' % (word, ' '.join([str(v) for v in model.wv[word]])))
logging.info('Done')

2023-11-14 13:22:54,277 : INFO : Save trained word vectors
100%|██████████| 135394/135394 [00:25<00:00, 5367.04it/s]
2023-11-14 13:23:23,972 : INFO : Done


In [7]:
print(emb_file)

/home/manni/embs/CDA_en_text8_mc3_epoch5_300.txt
